# Longitudinal Civic Engagement Survey — Data Merging Pipeline
**Authors:** Dr. Clarke De Reza & Dr. Grosse — Washington College  
**Purpose:** Merge three years of Qualtrics survey data (SP23, SP24, SP25) into a single longitudinal dataset following the tasks defined in the README.

---
**Tasks covered:**
1. Create a `survey_year` identifier variable  
2. Identify common variables across years and standardize names (using Qualtrics `ImportId` as the reliable cross-year key)  
3. Recode categorical variables consistently  
4. Recode open-response major/minor fields into WC 3-letter catalog codes (with double-major support)  
5. Align measurement scales (low = disagree, high = agree)  
6. Recode empty/missing cells to `-9` for SPSS  
7. Export merged CSV + codebook

## 1 · Import Required Libraries

In [56]:
import json
import re
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 60)
pd.set_option("display.width", 120)
print("pandas", pd.__version__, "  numpy", np.__version__)


pandas 3.0.1   numpy 2.4.2


## 2 · Load the Three Years of Survey Data

Qualtrics exports contain **three header rows** before actual responses begin:
- Row 0 (used as column names by pandas): short Qualtrics variable names, e.g. `Q16_7`
- Row index 0: full question text
- Row index 1: JSON ImportId, e.g. `{"ImportId":"QID21_7"}` — the **stable cross-year key**
- Row index 2+: respondent data

We read everything as strings and strip the two metadata rows before any processing.

In [57]:
FILE_MAP = {
    "SP23": "CE Data SP23 (1).csv",
    "SP24": "CE Data SP24.csv",
    "SP25": "CE Data SP25.csv",
}

raw = {}
question_text = {}   # col -> full question text (row 0)
import_id_map = {}   # col -> ImportId string (row 1)

for year, fname in FILE_MAP.items():
    df = pd.read_csv(fname, dtype=str, keep_default_na=False)
    question_text[year] = {col: str(df.loc[0, col]).strip() for col in df.columns}
    import_id_map[year] = {
        col: json.loads(str(df.loc[1, col].strip())).get("ImportId", "")
        for col in df.columns
    }
    # Drop the two Qualtrics metadata rows; reset index
    df = df.iloc[2:].reset_index(drop=True)
    raw[year] = df
    print(f"{year}: {df.shape[0]} responses, {df.shape[1]} columns")


SP23: 229 responses, 87 columns
SP24: 328 responses, 121 columns
SP25: 282 responses, 104 columns


## 3 · Create Year Identifier Variable

In [58]:
for year, df in raw.items():
    df.insert(0, "survey_year", year)

print("survey_year values added:", {yr: df["survey_year"].unique().tolist() for yr, df in raw.items()})


survey_year values added: {'SP23': ['SP23'], 'SP24': ['SP24'], 'SP25': ['SP25']}


## 4 · Identify and Standardize Common Variable Names

We use the Qualtrics **ImportId** (row 1) as the reliable cross-year identifier because surface-level column names shift across exports (e.g. `Q16_7` in SP23 maps to `Q16_8` in SP24/SP25 because a new sub-item was inserted).

**Key cross-year fixes confirmed from ImportId analysis:**

| ImportId | SP23 col | SP24 col | SP25 col | Canonical name |
|---|---|---|---|---|
| QID21_1…QID21_6 | Q16_1…Q16_6 | Q16_1…Q16_6 | Q16_1…Q16_6 | Q16_1…Q16_6 |
| QID21_7 | Q16_7 | Q16_8 | Q16_8 | Q16_7 |
| QID21_8 | Q16_8 | Q16_9 | Q16_9 | Q16_8 |
| QID21_9 | *(absent)* | Q16_7 | Q16_7 | *dropped* (not in SP23) |
| QID13 (YikYak) | Q11-M3 | *(absent)* | *(absent)* | *dropped* |
| QID16/QID14_*/QID15 | YikYak sub-qs | *(absent)* | *(absent)* | *dropped* |
| QID10_1/2/3 | Q9-ESP3_1/2/3 | Q9-ESP3_1/2/3 | *(absent)* | *dropped* |

**Questions only in SP24/SP25 (dropped):** Q50, Q51, Q52, Q54–Q59, Q60  
**Questions only in SP25 (dropped):** Q9-ESP3 (QID10 collapsed), Q10-ESP4 through Q14-ESP8, Q66, Q68–Q73, Q75, Q76  
**Questions only in SP24/SP25 but kept (filled -9 for SP23):** Q45 (minor, QID45_TEXT), Q46 (transfer status, QID46)

In [59]:
# ── Step A: build ImportId → (year → original_col) lookup ──────────────────
id_to_col = {}   # importId -> {year: col}
for year in FILE_MAP:
    for col, iid in import_id_map[year].items():
        if iid:
            id_to_col.setdefault(iid, {})[year] = col

# ── Step B: determine which ImportIds appear in ALL three years ─────────────
all_years = set(FILE_MAP.keys())

# Qualtrics system fields always kept regardless
SYSTEM_IMPORT_IDS = {
    "startDate", "endDate", "status", "ipAddress", "progress",
    "duration", "finished", "recordedDate", "_recordId",
    "recipientLastName", "recipientFirstName", "recipientEmail",
    "externalDataReference", "locationLatitude", "locationLongitude",
    "distributionChannel", "userLanguage",
}

# ImportIds to explicitly DROP even if present in all years
DROP_IMPORT_IDS = {
    "QID13",   # YikYak use (SP23 only)
    "QID16",   # YikYak connection (SP23 only)
    "QID14_1", "QID14_2", "QID14_3", "QID14_4", "QID14_5",
    "QID14_6", "QID14_7", "QID14_8", "QID14_9",  # YikYak impact (SP23 only)
    "QID15",   # YikYak as CE? (SP23 only)
    "QID21_9", # New Q16 sub-item in SP24/SP25 not in SP23
    # SP24/25-only blocks
    "QID50_1","QID50_2","QID50_3","QID50_4","QID50_5","QID50_6",
    "QID51_1","QID51_2","QID51_3","QID51_4","QID51_5","QID51_6",
    "QID52_1","QID52_2","QID52_3","QID52_4",
    "QID54_1","QID54_2","QID54_3","QID54_4",
    "QID55_1","QID55_2","QID55_3","QID55_4",
    "QID56_1","QID56_2","QID56_3","QID56_4",
    "QID57_1","QID57_2","QID57_3","QID57_4",
    "QID58","QID59_1","QID59_2","QID59_3","QID59_4",
    "QID60",
    # SP25-only blocks
    "QID10",   # collapsed version of QID10_1-3 (not comparable)
    "QID61","QID62","QID63","QID64","QID65","QID66",
    "QID68","QID69","QID70","QID71",
    "QID72_1","QID72_2","QID72_3","QID72_4",
    "QID73","QID75","QID76",
    # SP24-only membership sub-blocks (not in SP23 or SP25)
    "QID10_1","QID10_2","QID10_3",  # Q9-ESP3 sub-items absent in SP25
    "QID48","QID49",
    # Qualtrics AI topic analysis columns in SP24
    "QID45_TEXT_66a6fe78_c9z89r8",  # Q45 - Parent Topics / Topics hierarchy
}

# ImportIds present in SP24 & SP25 but NOT SP23 that we KEEP (fill SP23 with -9)
PARTIAL_KEEP_IDS = {
    "QID45_TEXT",  # minor / intended minor
    "QID46",       # transfer student
}

# Canonical names: ImportId -> final column name used in merged output
CANONICAL = {
    # Qualtrics system
    "startDate": "StartDate", "endDate": "EndDate", "status": "Status",
    "ipAddress": "IPAddress", "progress": "Progress", "duration": "Duration_sec",
    "finished": "Finished", "recordedDate": "RecordedDate", "_recordId": "ResponseId",
    "recipientLastName": "RecipientLastName", "recipientFirstName": "RecipientFirstName",
    "recipientEmail": "RecipientEmail",
    "externalDataReference": "ExternalReference",
    "locationLatitude": "LocationLatitude", "locationLongitude": "LocationLongitude",
    "distributionChannel": "DistributionChannel", "userLanguage": "UserLanguage",
    # Survey questions
    "QID37": "Consent",
    "QID2":  "CE_definition_text",
    "QID3":  "CE_define_open",
    "QID4_1": "CESA_1", "QID4_5": "CESA_2", "QID4_3": "CESA_3",
    "QID4_4": "CESA_4", "QID4_11": "CESA_5", "QID4_12": "CESA_6",
    "QID4_6": "CESA_7", "QID4_7": "CESA_8", "QID4_8": "CESA_9",
    "QID4_9": "CESA_10",
    "QID6":        "Community_ref",
    "QID6_4_TEXT": "Community_ref_other",
    "QID7":  "Volunteer_yn",
    "QID8":  "Volunteer_hrs_month",
    "QID9_1": "Civic_duty_1", "QID9_2": "Civic_duty_2", "QID9_3": "Civic_duty_3",
    "QID9_4": "Civic_duty_4", "QID9_5": "Civic_duty_5", "QID9_6": "Civic_duty_6",
    "QID31_1": "CE_activity_1", "QID31_2": "CE_activity_2",
    "QID31_3": "CE_activity_3", "QID31_4": "CE_activity_4",
    "QID31_5": "CE_activity_5", "QID31_6": "CE_activity_6",
    "QID11": "Media_CE_activities",
    "QID17": "Local_news_yn",
    "QID18": "Local_news_which",
    "QID19": "Local_news_connected",
    "QID20": "Local_news_CE_yn",
    "QID21_1": "CE_campus_1", "QID21_2": "CE_campus_2", "QID21_3": "CE_campus_3",
    "QID21_4": "CE_campus_4", "QID21_5": "CE_campus_5", "QID21_6": "CE_campus_6",
    "QID21_7": "CE_campus_7",   # SP23 Q16_7 / SP24-25 Q16_8
    "QID21_8": "CE_campus_8",   # SP23 Q16_8 / SP24-25 Q16_9
    "QID22": "Know_WC_CE_mission",
    "QID34": "Agree_CE_mission",
    "QID35": "WC_meeting_CE_mission",
    "QID36": "WC_CE_resources",
    "QID23": "Overall_CE_rating",
    "QID24": "Grad_year",
    "QID25_TEXT": "Major_raw",
    "QID26": "Athlete",
    "QID27": "Greek_life",
    "QID32": "Club_member",
    "QID28": "Gender",
    "QID29": "Race_ethnicity",
    "QID44_TEXT": "Participant_code",
    # Partial (SP24/SP25 only, filled -9 for SP23)
    "QID45_TEXT": "Minor_raw",
    "QID46": "Transfer_student",
}

# ── Step C: rename each year's DataFrame to canonical names ─────────────────
renamed = {}
for year, df in raw.items():
    rename_dict = {}
    for col in df.columns:
        if col == "survey_year":
            continue
        iid = import_id_map[year].get(col, "")
        if iid in CANONICAL:
            rename_dict[col] = CANONICAL[iid]
    df2 = df.rename(columns=rename_dict)
    renamed[year] = df2

# ── Step D: print mapping table for documentation ───────────────────────────
mapping_rows = []
for iid, canonical in CANONICAL.items():
    row = {"ImportId": iid, "Canonical": canonical}
    for year in FILE_MAP:
        row[year] = id_to_col.get(iid, {}).get(year, "—")
    mapping_rows.append(row)

mapping_df = pd.DataFrame(mapping_rows)
print("Variable mapping (ImportId → canonical name, by year):")
print(mapping_df.to_string(index=False))


Variable mapping (ImportId → canonical name, by year):
             ImportId             Canonical                  SP23                  SP24                  SP25
            startDate             StartDate             StartDate             StartDate             StartDate
              endDate               EndDate               EndDate               EndDate               EndDate
               status                Status                Status                Status                Status
            ipAddress             IPAddress             IPAddress             IPAddress             IPAddress
             progress              Progress              Progress              Progress              Progress
             duration          Duration_sec Duration (in seconds) Duration (in seconds) Duration (in seconds)
             finished              Finished              Finished              Finished              Finished
         recordedDate          RecordedDate          RecordedDate

In [60]:
# Keep only canonical columns that are either:
#   a) present in all 3 years, OR
#   b) in PARTIAL_KEEP_IDS (filled with -9 for SP23)

# Build set of canonical names that exist across all 3 years
canonical_in_year = {year: set(df.columns) for year, df in renamed.items()}
common_canonical = canonical_in_year["SP23"] & canonical_in_year["SP24"] & canonical_in_year["SP25"]

# Add partial-keep columns (keep even if not in SP23)
partial_canonical = {CANONICAL[iid] for iid in PARTIAL_KEEP_IDS if iid in CANONICAL}

# Final keep set = common + partial + survey_year
keep_cols = common_canonical | partial_canonical | {"survey_year"}

# Drop non-keep columns from each DataFrame and add missing partial cols as empty
filtered = {}
for year, df in renamed.items():
    cols_to_keep = [c for c in df.columns if c in keep_cols]
    df2 = df[cols_to_keep].copy()
    # Add partial columns for SP23 (they don't exist; will become -9 later)
    for pc in partial_canonical:
        if pc not in df2.columns:
            df2[pc] = ""
    filtered[year] = df2

print("Columns in final merged set:", sorted(keep_cols))
print("\nColumn counts per year:", {yr: len(df.columns) for yr, df in filtered.items()})


Columns in final merged set: ['Agree_CE_mission', 'CESA_1', 'CESA_10', 'CESA_2', 'CESA_3', 'CESA_4', 'CESA_5', 'CESA_6', 'CESA_7', 'CESA_8', 'CESA_9', 'CE_activity_1', 'CE_activity_2', 'CE_activity_3', 'CE_activity_4', 'CE_activity_5', 'CE_activity_6', 'CE_campus_1', 'CE_campus_2', 'CE_campus_3', 'CE_campus_4', 'CE_campus_5', 'CE_campus_6', 'CE_campus_7', 'CE_campus_8', 'CE_define_open', 'CE_definition_text', 'Civic_duty_1', 'Civic_duty_2', 'Civic_duty_3', 'Civic_duty_4', 'Civic_duty_5', 'Civic_duty_6', 'Community_ref', 'Community_ref_other', 'Consent', 'DistributionChannel', 'Duration_sec', 'EndDate', 'ExternalReference', 'Finished', 'Gender', 'Grad_year', 'IPAddress', 'Know_WC_CE_mission', 'Local_news_CE_yn', 'Local_news_connected', 'Local_news_which', 'Local_news_yn', 'LocationLatitude', 'LocationLongitude', 'Major_raw', 'Media_CE_activities', 'Minor_raw', 'Overall_CE_rating', 'Participant_code', 'Progress', 'Race_ethnicity', 'RecipientEmail', 'RecipientFirstName', 'RecipientLastNam

## 5 · Recode Categorical Variables Consistently

All categorical and ordinal variables are recoded to numeric so that:
- **Yes/No** → 1 / 0
- **Likert agreement** → 1 (Strongly disagree) … 5 (Strongly agree)
- **Likert frequency** → 1 (Never) … 5 (Very often / Always)
- **Likert connection** → 1 (Very disconnected) … 5 (Very connected)
- **Likert importance** (local news as CE) → 1 (Definitely not) … 4 (Definitely yes)
- **Class year** ordinal → 1 (First-year) … 5 (Graduate/Other)
- **Overall CE self-rating** → 1 (Not engaged) … 5 (Very engaged)

In [70]:
YES_NO_MAP = {
    "Yes": 1, "No": 0,
    "yes": 1, "no": 0,
    "TRUE": 1, "FALSE": 0,
    "True": 1, "False": 0,
}

# CESA scale: SP23 uses a 5-pt scale with "Somewhat agree/disagree"; SP24/25 similar.
# Mapping all observed labels → 1 (most disagree) … 5 (most agree).
AGREE5_MAP = {
    "Strongly agree": 5,    "Agree": 4,    "Somewhat agree": 4,
    "Neither agree nor disagree": 3,       "Neutral": 3,
    "Somewhat disagree": 2, "Disagree": 2,
    "Strongly disagree": 1,
    # capitalization variants
    "Strongly Agree": 5,    "Strongly Disagree": 1,
}

# Civic duty scale (Q8-ESP2) uses importance wording across all three years.
CIVIC_DUTY_MAP = {
    "Very Important": 4,    "Important": 3,
    "Not really important": 2,
    "Not at all important": 1,
    # fallback agree wording if it changes in future
    "Strongly agree": 5,    "Agree": 4,
    "Neither agree nor disagree": 3,
    "Disagree": 2,          "Strongly disagree": 1,
}

# Q16 / CE_campus scale — uses Strongly Agree / Agree / Neutral / Disagree / Strongly Disagree
CE_CAMPUS_MAP = AGREE5_MAP.copy()

# Volunteer yes/no (Q7)
VOLUNTEER_MAP = YES_NO_MAP.copy()

# Volunteer hours per month (Q7a) — ordinal text bands → integers
VOLUNTEER_HRS_MAP = {
    "Less than 4 hours per month":  1,
    "4 to 8 hours per month":       2,
    "8 to 12 hours per month":      3,
    "More than 12 hours per month": 4,
}

# Frequency scale for Q31 (In the last year, how often…)
# SP23/24/25 all use: Never / Sometimes/Occasionally / Often  (only 3 values observed)
FREQ5_MAP = {
    "Never": 1,
    "Rarely (once or twice a year)": 2,
    "Sometimes/Occasionally": 3,
    "Occasionally (a few times a year)": 3,
    "Often": 4,
    "Often (monthly)": 4,
    "Very often (weekly or more)": 5,
    # alternate short wordings
    "Rarely": 2, "Occasionally": 3, "Very often": 5,
}

# Connection scale (Q14-M6b local news)
CONNECTION_MAP = {
    "Very connected": 5, "Connected": 4,
    "Neither connected nor disconnected": 3,
    "Disconnected": 2, "Very disconnected": 1,
    "Neither connected or disconnected": 3,
}

# CE count/yes (Q15-M7 local news as CE)
CE_YN_MAP = {
    "Yes, definitely": 4, "Yes, probably": 3,
    "No, probably not": 2, "No, definitely not": 1,
    "Yes": 4, "No": 1,
}

# Community reference (Q6)
COMMUNITY_REF_MAP = {
    "The Chestertown community": 1,
    "The Chestertown or local community": 1,
    "Chestertown community": 1,
    "The Washington College community": 2,
    "The broader national community": 3,
    "National community": 3,
    "Other (please specify)": 4,
    "Somewhere else (please specify)": 4,
    "Other": 4,
    "Your home community": 5,
}

# Graduation year (Q19)
GRAD_YEAR_MAP = {
    "2023": 2023, "2024": 2024, "2025": 2025, "2026": 2026,
    "2027": 2027, "2028": 2028, "2029": 2029, "2030": 2030,
}

# Overall CE self-rating (Q18) — comparative phrasing used across all years
CE_RATING_MAP = {
    "I am much more engaged than the average WAC student": 5,
    "I am more engaged than the average WAC student": 4,
    "I am as engaged as the average WAC student": 3,
    "I am less engaged than the average WAC student": 2,
    "I am much less engaged than the average WAC student": 1,
    # simple fallbacks
    "Not at all engaged": 1, "Slightly engaged": 2, "Moderately engaged": 3,
    "Very engaged": 4,       "Extremely engaged": 5, "Not engaged": 1,
}

# Mission knowledge (Q17) — Yes/No
MISSION_YN_MAP = YES_NO_MAP.copy()

# Agree CE should be in mission (Q34) — Yes / No / I don't know
AGREE_MISSION_MAP = {
    "Yes": 1, "No": 0, "I don't know": -9,
    "yes": 1, "no": 0,
}

# Meeting mission (Q35), providing resources (Q36) — Yes / No / I don't know
MEETING_MISSION_MAP = {
    "Yes": 2, "I don't know": 1, "No": 0,
    "Somewhat": 1,        # alternate wording
    "yes": 2, "no": 0,
}

# Athlete / greek / club — yes/no
ATHLETE_MAP   = YES_NO_MAP.copy()
GREEK_MAP     = YES_NO_MAP.copy()
CLUB_MAP      = YES_NO_MAP.copy()
TRANSFER_MAP  = YES_NO_MAP.copy()

# Gender — reconcile labels across years
# SP23/24 use Male/Female; SP25 uses Man/Woman; all use genderqueer, transgender, etc.
GENDER_MAP = {
    "Man": 1, "Male": 1,
    "Woman": 2, "Female": 2,
    "Genderqueer, genderfluid, or non-binary": 3,
    "Non-binary / third gender": 3, "Non-binary": 3,
    "Transgender": 4,
    "I identify using different terms": 5,
    "Prefer to self-describe": 5,
    "I prefer not to answer": 6,
    "Prefer not to say": 6,
    "Volunteer_hrs_month": VOLUNTEER_HRS_MAP,
    "Local_news_yn":      YES_NO_MAP,
    "Know_WC_CE_mission": MISSION_YN_MAP,
    "Agree_CE_mission":   AGREE_MISSION_MAP,
    "Athlete":            ATHLETE_MAP,
    "Greek_life":         GREEK_MAP,
    "Club_member":        CLUB_MAP,
    "Transfer_student":   TRANSFER_MAP,
    # CESA — 5-pt agree/disagree (with "Somewhat" variants)
    "CESA_1":"AGREE5", "CESA_2":"AGREE5", "CESA_3":"AGREE5",
    "CESA_4":"AGREE5", "CESA_5":"AGREE5", "CESA_6":"AGREE5",
    "CESA_7":"AGREE5", "CESA_8":"AGREE5", "CESA_9":"AGREE5",
    "CESA_10":"AGREE5",
    # Civic duty — importance scale (Very Important … Not at all important)
    "Civic_duty_1":"CIVIC_DUTY", "Civic_duty_2":"CIVIC_DUTY", "Civic_duty_3":"CIVIC_DUTY",
    "Civic_duty_4":"CIVIC_DUTY", "Civic_duty_5":"CIVIC_DUTY", "Civic_duty_6":"CIVIC_DUTY",
    # CE campus — Strongly Agree / Agree / Neutral / Disagree / Strongly Disagree
    "CE_campus_1":"AGREE5", "CE_campus_2":"AGREE5", "CE_campus_3":"AGREE5",
    "CE_campus_4":"AGREE5", "CE_campus_5":"AGREE5", "CE_campus_6":"AGREE5",
    "CE_campus_7":"AGREE5", "CE_campus_8":"AGREE5",
    # Frequency — Never / Sometimes/Occasionally / Often
    "CE_activity_1":"FREQ5", "CE_activity_2":"FREQ5", "CE_activity_3":"FREQ5",
    "CE_activity_4":"FREQ5", "CE_activity_5":"FREQ5", "CE_activity_6":"FREQ5",
    # Connection
    "Local_news_connected": CONNECTION_MAP,
    # CE count
    "Local_news_CE_yn": CE_YN_MAP,
    "WC_meeting_CE_mission": MEETING_MISSION_MAP,
    "WC_CE_resources": MEETING_MISSION_MAP,
    # Other
    "Community_ref":    COMMUNITY_REF_MAP,
    "Overall_CE_rating": CE_RATING_MAP,
    "Grad_year":        GRAD_YEAR_MAP,
    "Gender":           GENDER_MAP,
}

NAMED_MAPS = {"AGREE5": AGREE5_MAP, "FREQ5": FREQ5_MAP, "CIVIC_DUTY": CIVIC_DUTY_MAP}

recoded = {}
for year, df in filtered.items():
    df2 = df.copy()
    for col, rmap in COLUMN_RECODE_MAP.items():
        if col not in df2.columns:
            continue
        actual_map = NAMED_MAPS.get(rmap, rmap) if isinstance(rmap, str) else rmap
        df2[col] = df2[col].map(lambda v, m=actual_map: m.get(str(v).strip(), v))
    recoded[year] = df2

# Preview recode results
sample_cols = ["Volunteer_yn","CESA_1","Civic_duty_1","CE_campus_1",
               "CE_activity_1","Local_news_yn","Gender","Athlete","Overall_CE_rating"]
preview = pd.concat([recoded[y][["survey_year"] + [c for c in sample_cols if c in recoded[y].columns]].head(3)
                     for y in FILE_MAP])
print(preview.to_string(index=False))

survey_year Volunteer_yn CESA_1 Civic_duty_1 CE_campus_1 CE_activity_1 Local_news_yn Gender Overall_CE_rating
       SP23            1      2            4           5             4             0                        1
       SP23            1      4            1           5                           0      2                 2
       SP23            1      4            3           3                           0      2                 3
       SP24                                                                                                  
       SP24            1      2            4           5             4             1      2                 5
       SP24            1      4            3           2             3             0      1                 4
       SP25            1      4            4           4             3             1      3                 3
       SP25            1      4            3           3             3             0      1                 3
       SP2

## 6 · Recode Open-Response Major/Minor into WC Catalog Codes

The `Major_raw` field (QID25_TEXT) is a free-text entry. Respondents sometimes enter:
- A single major: `"Psychology"` → `PSY`
- Two majors separated by `/`, `&`, `and`, `,`, or a newline: `"English/History"` → `ENG`, `HIS`
- A minor combined with the major entry

We parse the raw text into up to two major codes (`Major1`, `Major2`) and one minor code (`Minor1`).  
The mapping uses the **Washington College course-catalog three-letter (or four-letter) prefixes**.

In [71]:
# ── Washington College catalog codes ────────────────────────────────────────
# Each entry: (regex_pattern_lowercase, canonical_code)
# Patterns are tried in order; first match wins.
WC_MAJOR_PATTERNS = [
    # Exact / near-exact catalog codes first
    (r"\bams\b|american\s+stud",               "AMS"),   # American Studies
    (r"\bant\b|anthropolog|\banthro\b",          "ANT"),
    (r"\bart\b|studio art|fine art|bfa\b|art\s+hist", "ART"),  # Art History → ART
    (r"\bbio\b|biolog",                        "BIO"),
    (r"\bneu\b|neurosci",                      "NEU"),   # Neuroscience
    (r"\bbus\b|\bbusiness\b|management",       "BUS"),
    (r"\bche\b|chemistr|\bchemical",             "CHE"),
    (r"\bcms\b|communicat|media stud|communications", "CMS"),
    (r"\bcsi\b|computer sci|comp sci|compsci", "CSI"),
    (r"\bdan\b|dance",                         "DAN"),
    (r"\beco\b|econom",                        "ECO"),
    # Human Development (WC HDS/HGD program) — before EDU to avoid false match
    (r"human\s+development|human\s+growth|human\s+dev|hds\b|hgd\b",  "HDS"),
    (r"\bedu\b|educat",                        "EDU"),
    (r"\beng\b|english|literature",            "ENG"),
    (r"\bens\b|environmental\s+stud",          "ENS"),
    (r"\benv\b|environmental\s+sci",           "ENV"),
    (r"\bfre\b|french",                        "FRE"),
    (r"\bgeo\b|geolog|geograph",               "GEO"),
    (r"\bgrs\b|germanic|\bgerman\b",             "GRS"),   # Germanic / German Studies
    (r"\bhis\b|histor",                        "HIS"),
    (r"\bhmn\b|humanities",                    "HMN"),   # Humanities
    (r"\bhps\b|hispanic\s+stud|hispanic",      "HPS"),   # Hispanic Studies
    (r"\bids\b|interdisciplin",                "IDS"),
    (r"\bint\b|international",                 "INT"),
    (r"\bmat\b|math",                          "MAT"),
    (r"\bmfce\b|museum.*field|chesapeake\s+regional",  "MFCE"),  # WC-specific
    (r"\bmus\b|music",                         "MUS"),
    (r"\bnsc\b|natural\s+sci",                 "NSC"),
    (r"\bphi\b|philosoph",                     "PHI"),
    (r"\bphy\b|physic(?!s\s*and\s*psycho)",    "PHY"),  # avoid 'physics and psychology'
    (r"\bpol\b|politic",                        "POL"),
    (r"\bpsy\b|psycholog|\bpsych\b|\bpsyc\b|clinical|counseling", "PSY"),
    (r"\brel\b|religio",                       "REL"),
    (r"\bsoc\b|sociolog",                      "SOC"),
    (r"\bspa\b|spanish",                       "SPA"),
    (r"\bthe\b|theatre|theater",               "THE"),
    (r"\bwgs\b|women.*gender|gender.*sexualit|women.*stud", "WGS"),
    (r"\bwrg\b|writing",                       "WRG"),
    # Social work / welfare catch
    (r"social\s+work|social\s+welfare",        "SOC"),
    (r"\bpre.?med\b|pre.?law\b",               "UNDECLARED"),
    (r"\bundeclar|undecided|unsure|tbd|n/?a\b|none|i\s+don.t\s+know", "UNDECLARED"),
]

def match_wc_code(text: str) -> str:
    """Return the first matching WC catalog code for a lowercased text fragment."""
    t = text.lower().strip()
    if not t:
        return ""
    # Check if the text IS already a known code (uppercase 2-4 letters)
    if re.fullmatch(r"[A-Z]{2,4}", text.strip()):
        return text.strip().upper()
    for pattern, code in WC_MAJOR_PATTERNS:
        if re.search(pattern, t):
            return code
    # Return a cleaned version of the unknown text for manual review
    return "OTHER:" + text.strip()[:20].upper().replace(" ", "_")


SEPARATORS = re.compile(
    r"\s*/\s*|\s+&\s+|;\s*|,\s*(?=\S)|\n+|\+",
    re.IGNORECASE,
)

# Secondary splitter for 'and'/'or' — applied first to detect multi-program entries
# (e.g. "Economics and Political Science" → ECO + POL), but only when splitting
# yields 2+ known codes; otherwise falls back to full-phrase match
# (e.g. "Human Growth and Development" → HDS as a single unit).
AND_SPLITTER = re.compile(r"\s+and\s+|\s+or\s+", re.IGNORECASE)

# Keywords that indicate the part after them is the MINOR, not a second major
MINOR_KEYWORDS = re.compile(
    r"\bminor\b|\bminoring\b|\bwith\s+a\s+minor\b",
    re.IGNORECASE,
)

def parse_major_minor(raw: str):
    # Parse a free-text major/minor entry.
    # Returns: (major1, major2, inferred_minor)
    #   major1, major2: WC catalog codes (or "" if absent)
    #   inferred_minor: code inferred from text like 'PSY with a minor in HIS'
    if not isinstance(raw, str) or not raw.strip():
        return "", "", ""

    text = raw.strip()

    # Split on 'minor' keyword first to separate intended minor text
    inferred_minor = ""
    minor_match = MINOR_KEYWORDS.search(text)
    if minor_match:
        minor_part = text[minor_match.end():].strip().lstrip("in").strip()
        text = text[:minor_match.start()].strip()
        inferred_minor = match_wc_code(minor_part)

    # Primary split on /  ;  ,  newline  +
    parts = [p.strip() for p in SEPARATORS.split(text) if p.strip()]

    # For each fragment: if it contains 'and/or', first try splitting to detect
    # multiple programs (e.g. "Economics and Political Science" → ECO + POL).
    # Fall back to full-phrase match when splitting doesn't yield 2+ known codes
    # (e.g. "Human Growth and Development" → HDS as a single unit).
    resolved = []
    for part in parts:
        if AND_SPLITTER.search(part):
            sub_parts = [s.strip() for s in AND_SPLITTER.split(part) if s.strip()]
            sub_codes = [match_wc_code(s) for s in sub_parts]
            known_subs = [c for c in sub_codes if c and not c.startswith("OTHER:")]
            if len(known_subs) >= 2:
                # Multiple distinct programs — use the split-derived codes
                resolved.extend(known_subs)
                continue
        # Single program (or compound name like "Human Growth and Development")
        resolved.append(match_wc_code(part))

    # De-duplicate while preserving order
    seen = []
    for m in resolved:
        if m and m not in seen:
            seen.append(m)

    major1 = seen[0] if len(seen) > 0 else ""
    major2 = seen[1] if len(seen) > 1 else ""
    if len(seen) > 2:
        major2 = major2 + "+" + "+".join(seen[2:])
    return major1, major2, inferred_minor


def parse_minor_field(raw: str) -> str:
    """Parse the dedicated minor text field (Q45) into a WC catalog code."""
    if not isinstance(raw, str) or not raw.strip():
        return ""
    parts = [p.strip() for p in SEPARATORS.split(raw.strip()) if p.strip()]
    codes = [match_wc_code(p) for p in parts if p]
    return "; ".join(c for c in codes if c)


# ── Apply to each year ───────────────────────────────────────────────────────
for year, df in recoded.items():
    parsed = df["Major_raw"].apply(parse_major_minor)
    df["Major1"] = [r[0] for r in parsed]
    df["Major2"] = [r[1] for r in parsed]
    df["Major_minor_inferred"] = [r[2] for r in parsed]  # minor hinted in major field

    # Parse the dedicated minor field if present (SP24/SP25); SP23 will be ""
    minor_raw_col = "Minor_raw"
    if minor_raw_col in df.columns:
        df["Minor1"] = df[minor_raw_col].apply(parse_minor_field)
    else:
        df["Minor1"] = ""

    # Prefer explicit Minor1 over inferred; merge them
    df["Minor1"] = df.apply(
        lambda row: row["Minor1"] if row["Minor1"] else row["Major_minor_inferred"],
        axis=1,
    )

# Preview
sample = pd.concat([recoded[y][["survey_year","Major_raw","Major1","Major2","Minor1"]].head(5)
                    for y in FILE_MAP])
print(sample.to_string(index=False))


survey_year                        Major_raw Major1 Major2                               Minor1
       SP23                                                                                    
       SP23                Human Development    HDS                                            
       SP23               Human Development     HDS                                            
       SP23               Human Development     HDS                                            
       SP23                Human Development    HDS                                            
       SP24                                                                                    
       SP24    Human Growth and Development     HDS                    OTHER:ELEMENTARY_ED_CERT
       SP24      Business Management and CMS    BUS    CMS                           UNDECLARED
       SP24                                                                                    
       SP24                Political Sci

In [72]:
# ── Diagnostic: show unmatched major entries for manual review ───────────────
all_major_raw = pd.concat(
    [recoded[y][["survey_year","Major_raw","Major1","Major2"]].copy() for y in FILE_MAP]
)
unmatched = all_major_raw[
    all_major_raw["Major1"].str.startswith("OTHER:", na=False) |
    all_major_raw["Major2"].str.startswith("OTHER:", na=False)
][["survey_year","Major_raw","Major1","Major2"]].drop_duplicates()

print(f"Entries with unmatched major text ({len(unmatched)} rows):")
print(unmatched.to_string(index=False))


Entries with unmatched major text (3 rows):
survey_year        Major_raw                 Major1 Major2
       SP23 European Studies OTHER:EUROPEAN_STUDIES       
       SP23     Lycanthropy       OTHER:LYCANTHROPY       
       SP24             swim             OTHER:SWIM       


## 7 · Align Measurement Scales

Convention: **low numbers = disagreement/less**, **high numbers = agreement/more**.

After the recoding above, all agree-type Likert variables run 1 (Strongly disagree) → 5 (Strongly agree). We verify this and flag any CESA or civic-duty item where the scale direction may be reversed (negative-valence items need reverse-scoring before interpretation, but per the README we standardize direction by keeping 1=disagree throughout—reverse items are noted in the codebook, not flipped here unless the raw scale was entered backwards).

In [73]:
# Verify that numeric Likert columns only contain expected values after recoding
LIKERT5_COLS = (
    [f"CESA_{i}" for i in range(1, 11)]
    + [f"Civic_duty_{i}" for i in range(1, 7)]
    + [f"CE_campus_{i}" for i in range(1, 9)]
    + [f"CE_activity_{i}" for i in range(1, 7)]
)

joint = pd.concat([recoded[y] for y in FILE_MAP], ignore_index=True)

print("=== Unique values per Likert column (should be 1–5 and any unconverted strings) ===")
issues = []
for col in LIKERT5_COLS:
    if col not in joint.columns:
        continue
    numeric_vals = pd.to_numeric(joint[col], errors="coerce")
    non_numeric = joint[col][(numeric_vals.isna()) & (joint[col].str.strip() != "")].unique()
    out_of_range = numeric_vals[(numeric_vals.notna()) & (~numeric_vals.isin([1,2,3,4,5]))].unique()
    if len(non_numeric) > 0 or len(out_of_range) > 0:
        issues.append({"column": col, "non_numeric": non_numeric.tolist(), "out_of_range": out_of_range.tolist()})

if issues:
    print("⚠ Columns with unexpected values:")
    for issue in issues:
        print(f"  {issue['column']}: non-numeric={issue['non_numeric']}, out_of_range={issue['out_of_range']}")
else:
    print("✓ All Likert columns contain only values 1–5 (plus empty strings that will become -9)")

# Scale check for Yes/No columns
YESNO_COLS = ["Volunteer_yn","Local_news_yn","Know_WC_CE_mission","Agree_CE_mission",
              "Athlete","Greek_life","Club_member","Transfer_student"]
print("\n=== Yes/No column value sets ===")
for col in YESNO_COLS:
    if col in joint.columns:
        print(f"  {col}: {sorted(joint[col].dropna().unique().tolist(), key=str)}")


=== Unique values per Likert column (should be 1–5 and any unconverted strings) ===
✓ All Likert columns contain only values 1–5 (plus empty strings that will become -9)

=== Yes/No column value sets ===
  Volunteer_yn: ['', 0, 1]
  Local_news_yn: ['', 0, 1]
  Know_WC_CE_mission: ['', 0, 1]
  Agree_CE_mission: ['', -9, 0, 1]
  Transfer_student: ['', 0, 1]


## 8 · Recode Missing Values to -9

Per the README, empty or NaN cells are recoded to `-9` so they can be designated as discrete missing values in SPSS (`MISSING VALUES varname (-9)`).

Cells are considered missing if they are:
- `NaN` / `None`  
- Empty string `""`  
- Qualtrics skip-logic sentinels like `"N/A"`, `"n/a"`

In [74]:
MISSING_SENTINELS = {"", "nan", "none", "n/a", "na", "N/A", "NA"}

def recode_missing(val):
    """Replace empty / NaN / sentinel values with -9."""
    if val is None:
        return -9
    s = str(val).strip()
    if s.lower() in {v.lower() for v in MISSING_SENTINELS}:
        return -9
    return val

filled = {}
for year, df in recoded.items():
    df2 = df.copy()
    # Only apply to non-system columns that are meaningful survey responses
    skip_cols = {"survey_year", "ResponseId", "StartDate", "EndDate",
                 "RecordedDate", "IPAddress", "RecipientEmail",
                 "RecipientFirstName", "RecipientLastName",
                 "ExternalReference", "Participant_code"}
    recode_cols = [c for c in df2.columns if c not in skip_cols]
    df2[recode_cols] = df2[recode_cols].map(recode_missing)
    filled[year] = df2

# Summary count of -9 per column
merged_preview = pd.concat([filled[y] for y in FILE_MAP], ignore_index=True)
missing_counts = (merged_preview == -9).sum().sort_values(ascending=False)
print("Columns with most -9 (missing) values:")
print(missing_counts[missing_counts > 0].to_string())


Columns with most -9 (missing) values:
Major_minor_inferred     839
CE_definition_text       839
Community_ref_other      807
Major2                   712
Local_news_which         666
Local_news_connected     665
Minor_raw                591
Minor1                   566
Transfer_student         502
Volunteer_hrs_month      493
Agree_CE_mission         430
WC_meeting_CE_mission    369
Major_raw                369
Major1                   369
WC_CE_resources          369
Race_ethnicity           362
Gender                   361
Grad_year                357
Overall_CE_rating        353
Know_WC_CE_mission       350
Media_CE_activities      341
CE_campus_3              340
CE_campus_2              340
CE_campus_6              340
CE_campus_7              340
CE_campus_1              340
CE_campus_4              340
CE_campus_5              340
CE_campus_8              340
Local_news_yn            328
Local_news_CE_yn         327
CE_activity_2            325
CE_activity_4            325
CE_a

## 9 · One-Hot Encode Multi-Select Responses

Three survey questions allow respondents to select multiple options. We one-hot encode each into binary indicator columns:
- **`CE_define_open`** — 13 civic engagement definition items
- **`Media_CE_activities`** — 7 social-media CE activities (near-duplicate labels across years are normalized by lowercasing and stripping trailing punctuation)
- **`Local_news_which`** — 6 local news outlets

Encoding convention: `1` = selected, `0` = not selected, `-9` = question was missing.  
Original raw columns are retained; new columns are appended.

In [75]:
# ── Option → column-name maps ─────────────────────────────────────────────
# Each entry: (substring to match in lowercased response, output column name)
# Matching is case-insensitive substring; trailing periods stripped before matching.

CE_DEFINE_OHE = [
    ("voting",                                            "CEdef_voting"),
    ("volunteering or doing community service",           "CEdef_volunteering"),
    ("talking with people in the community",              "CEdef_talking_community"),
    ("activism or awareness raising",                     "CEdef_activism"),
    ("political engagement",                              "CEdef_political"),
    ("protesting or rallying",                            "CEdef_protesting"),
    ("teaching or mentorship",                            "CEdef_teaching"),
    ("fundraising",                                       "CEdef_fundraising"),
    ("attending community arts and culture",              "CEdef_arts_culture"),
    ("contributing to community businesses",              "CEdef_community_business"),
    ("making donations to charities",                     "CEdef_donations"),
    ("participating in research with community",          "CEdef_research"),
    ("service-learning experiences",                      "CEdef_service_learning"),
]

MEDIA_OHE = [
    ("reposting information",                            "Media_repost_events"),
    ("signing online petitions",                         "Media_sign_petition"),
    ("following accounts/pages that post about current", "Media_follow_news"),
    ("following elected officials",                      "Media_follow_officials"),
    ("debating opinions",                                "Media_debate_opinions"),
    ("liking posts",                                     "Media_like_posts"),
    ("posting your own opinions",                        "Media_post_opinions"),
]

NEWS_OHE = [
    ("the elm",               "News_The_Elm"),
    ("kent county news",      "News_Kent_County"),
    ("the chestertown spy",   "News_Chestertown_Spy"),
    ("eastern shore post",    "News_Eastern_Shore_Post"),
    ("shore daily news",      "News_Shore_Daily"),
    ("delmarva now",          "News_Delmarva_Now"),
]

def one_hot_multiselect(series, option_specs):
    """
    Given a Series of comma-separated multi-select responses (or -9 for missing),
    return a DataFrame with one binary column per option.
      1  = option was selected
      0  = option was not selected (question was answered)
     -9  = question was skipped / missing
    """
    result = {}
    for substr, col_name in option_specs:
        col_vals = []
        for val in series:
            if val == -9 or str(val).strip() == "-9":
                col_vals.append(-9)
            else:
                # Normalize: lowercase, strip trailing period
                normalized = str(val).lower().rstrip(".")
                col_vals.append(1 if substr.lower() in normalized else 0)
        result[col_name] = col_vals
    return pd.DataFrame(result, index=series.index)

# Apply to each year's filled DataFrame so columns are present before concat
for year, df in filled.items():
    cedef_ohe = one_hot_multiselect(df["CE_define_open"],      CE_DEFINE_OHE)
    media_ohe = one_hot_multiselect(df["Media_CE_activities"], MEDIA_OHE)
    news_ohe  = one_hot_multiselect(df["Local_news_which"],    NEWS_OHE)
    for col in cedef_ohe.columns:
        df[col] = cedef_ohe[col]
    for col in media_ohe.columns:
        df[col] = media_ohe[col]
    for col in news_ohe.columns:
        df[col] = news_ohe[col]

# Preview CE_define_open OHE
cedef_ohe_cols = [s[1] for s in CE_DEFINE_OHE]
preview_cedef = pd.concat([
    filled[y][["survey_year", "CE_define_open"] + cedef_ohe_cols].head(2)
    for y in FILE_MAP
])
print("CE_define_open one-hot preview:")
print(preview_cedef.to_string(index=False))

# Preview Media OHE
ohe_cols = [s[1] for s in MEDIA_OHE]
preview_ohe = pd.concat([
    filled[y][["survey_year", "Media_CE_activities"] + ohe_cols].head(3)
    for y in FILE_MAP
])
print("\nMedia_CE_activities one-hot preview:")
print(preview_ohe.to_string(index=False))

# Preview News OHE
news_ohe_cols = [s[1] for s in NEWS_OHE]
preview_news = pd.concat([
    filled[y][["survey_year", "Local_news_which"] + news_ohe_cols].head(3)
    for y in FILE_MAP
])
print("\nLocal_news_which one-hot preview:")
print(preview_news.to_string(index=False))


CE_define_open one-hot preview:
survey_year                                                                                                                                                                                                                                                                                                                                                                                                                            CE_define_open  CEdef_voting  CEdef_volunteering  CEdef_talking_community  CEdef_activism  CEdef_political  CEdef_protesting  CEdef_teaching  CEdef_fundraising  CEdef_arts_culture  CEdef_community_business  CEdef_donations  CEdef_research  CEdef_service_learning
       SP23                                                                                                                                                                                                                                                                               

## 10 · Export Merged Dataset

Concatenate all three years into one longitudinal DataFrame, order columns logically, and export to both CSV and Excel.

In [77]:
# Concatenate all three years
merged = pd.concat([filled[y] for y in FILE_MAP], ignore_index=True)

# ── Reorder columns: year → ID → demographics → survey items ─────────────────
ID_COLS   = ["survey_year", "ResponseId", "StartDate", "RecordedDate",
             "Status", "Finished", "Duration_sec", "Progress",
             "DistributionChannel"]
DEMO_COLS = ["Grad_year", "Major_raw", "Major1", "Major2",
             "Transfer_student", "Athlete", "Greek_life", "Club_member",
             "Gender", "Race_ethnicity", "Participant_code"]
# All remaining survey columns in original order
OTHER_COLS = [c for c in merged.columns
              if c not in ID_COLS + DEMO_COLS
              and c not in {"RecipientLastName","RecipientFirstName",
                            "RecipientEmail","ExternalReference",
                            "IPAddress","LocationLatitude","LocationLongitude",
                            "UserLanguage",
                            "Minor_raw","Minor1","Major_minor_inferred"}]

final_cols = (
    [c for c in ID_COLS   if c in merged.columns] +
    [c for c in DEMO_COLS if c in merged.columns] +
    [c for c in OTHER_COLS if c in merged.columns]
)
merged = merged[final_cols]

print(f"Merged dataset shape: {merged.shape}")
print(f"Years present: {merged['survey_year'].value_counts().to_dict()}")

# Export
OUT_CSV   = "CE_longitudinal_merged.csv"
OUT_EXCEL = "CE_longitudinal_merged.xlsx"

merged.to_csv(OUT_CSV, index=False)
merged.to_excel(OUT_EXCEL, index=False, sheet_name="Merged_Data")

print(f"\n✓ Saved: {OUT_CSV}")
print(f"✓ Saved: {OUT_EXCEL}")


Merged dataset shape: (839, 91)
Years present: {'SP24': 328, 'SP25': 282, 'SP23': 229}

✓ Saved: CE_longitudinal_merged.csv
✓ Saved: CE_longitudinal_merged.xlsx


## 11 · Generate Codebook

A comprehensive codebook is generated programmatically documenting each variable's:
- Canonical name  
- Original Qualtrics column name (by year)  
- Full question text  
- Data type  
- Valid values / coding scheme  
- Notes on recoding decisions  
- `-9` = Missing value sentinel

In [78]:
# Reverse mapping: canonical_name → ImportId
CANONICAL_INV = {v: k for k, v in CANONICAL.items()}

def get_question_text(canonical_col):
    """Retrieve the full question text for a canonical column from any available year."""
    iid = CANONICAL_INV.get(canonical_col)
    if not iid:
        return ""
    for year in FILE_MAP:
        for col, cid in import_id_map[year].items():
            if cid == iid:
                return question_text[year].get(col, "")[:300].replace("\n", " ")
    return ""

def get_orig_cols(canonical_col):
    """Return 'SP23: col / SP24: col / SP25: col' for a canonical column."""
    iid = CANONICAL_INV.get(canonical_col)
    if not iid:
        return ""
    parts = []
    for year in FILE_MAP:
        orig = id_to_col.get(iid, {}).get(year, "—")
        parts.append(f"{year}:{orig}")
    return " / ".join(parts)

# Valid-values & notes for each canonical variable
CODEBOOK_META = {
    "survey_year":        ("string", "SP23 / SP24 / SP25", "Added: identifies data source year"),
    "Consent":            ("binary", "1=Agreed, 0=Not agreed", "Qualtrics consent gate"),
    "CE_definition_text": ("text", "Displayed text passage", "Civic engagement definition shown to respondent"),
    "CE_define_open":     ("text", "Open response", "Respondent's own definition of civic engagement"),
    "CESA_1":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 1"),
    "CESA_2":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 2"),
    "CESA_3":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 3"),
    "CESA_4":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 4"),
    "CESA_5":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 5"),
    "CESA_6":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 6"),
    "CESA_7":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 7"),
    "CESA_8":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 8"),
    "CESA_9":             ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 9"),
    "CESA_10":            ("Likert-5", "1=Strongly disagree…5=Strongly agree", "CESA item 10"),
    "Community_ref":      ("categorical", "1=Chestertown, 2=WC, 3=National, 4=Other, 5=Home", "Which community respondent referenced for prior Qs"),
    "Community_ref_other":("text", "Open response", "Specified 'other' community"),
    "Volunteer_yn":       ("binary", "1=Yes, 0=No", "Volunteer in community?"),
    "Volunteer_hrs_month":("ordinal", "1=<4 hrs, 2=4-8 hrs, 3=8-12 hrs, 4=>12 hrs", "Avg volunteer hrs/month (ordinal bands)"),
    "Civic_duty_1":       ("Likert-5", "1=SD…5=SA", "Civic duty scale item 1"),
    "Civic_duty_2":       ("Likert-5", "1=SD…5=SA", "Civic duty scale item 2"),
    "Civic_duty_3":       ("Likert-5", "1=SD…5=SA", "Civic duty scale item 3"),
    "Civic_duty_4":       ("Likert-5", "1=SD…5=SA", "Civic duty scale item 4"),
    "Civic_duty_5":       ("Likert-5", "1=SD…5=SA", "Civic duty scale item 5"),
    "Civic_duty_6":       ("Likert-5", "1=SD…5=SA", "Civic duty scale item 6"),
    "CE_activity_1":      ("Likert-5 frequency", "1=Never…5=Very often (weekly+)", "CE activity frequency item 1"),
    "CE_activity_2":      ("Likert-5 frequency", "1=Never…5=Very often (weekly+)", "CE activity frequency item 2"),
    "CE_activity_3":      ("Likert-5 frequency", "1=Never…5=Very often (weekly+)", "CE activity frequency item 3"),
    "CE_activity_4":      ("Likert-5 frequency", "1=Never…5=Very often (weekly+)", "CE activity frequency item 4"),
    "CE_activity_5":      ("Likert-5 frequency", "1=Never…5=Very often (weekly+)", "CE activity frequency item 5"),
    "CE_activity_6":      ("Likert-5 frequency", "1=Never…5=Very often (weekly+)", "CE activity frequency item 6"),
    "Media_CE_activities":("multi-select text", "Comma-separated selected options", "Social media activities seen as CE (Q10-M1)"),
    "Local_news_yn":      ("binary", "1=Yes, 0=No", "Reads local news regularly?"),
    "Local_news_which":   ("multi-select text", "Comma-separated publications", "Which local publications read"),
    "Local_news_connected":("Likert-5 connection", "1=Very disconnected…5=Very connected", "Connection from reading local news"),
    "Local_news_CE_yn":   ("Likert-4", "1=Definitely not…4=Definitely yes", "Does reading local news count as CE?"),
    "CE_campus_1":        ("Likert-5", "1=SD…5=SA", "CE campus experience Q16 item 1 (QID21_1)"),
    "CE_campus_2":        ("Likert-5", "1=SD…5=SA", "CE campus experience Q16 item 2 (QID21_2)"),
    "CE_campus_3":        ("Likert-5", "1=SD…5=SA", "CE campus experience Q16 item 3 (QID21_3)"),
    "CE_campus_4":        ("Likert-5", "1=SD…5=SA", "CE campus experience Q16 item 4 (QID21_4)"),
    "CE_campus_5":        ("Likert-5", "1=SD…5=SA", "CE campus experience Q16 item 5 (QID21_5)"),
    "CE_campus_6":        ("Likert-5", "1=SD…5=SA", "CE campus experience Q16 item 6 (QID21_6)"),
    "CE_campus_7":        ("Likert-5", "1=SD…5=SA", "CE campus item QID21_7: SP23=Q16_7, SP24/25=Q16_8 (renumbered due to new Q16_7 in SP24+)"),
    "CE_campus_8":        ("Likert-5", "1=SD…5=SA", "CE campus item QID21_8: SP23=Q16_8, SP24/25=Q16_9 (renumbered similarly)"),
    "Know_WC_CE_mission": ("binary", "1=Yes, 0=No", "Knew WC had CE in its mission?"),
    "Agree_CE_mission":   ("binary", "1=Yes, 0=No", "Agrees CE should be in mission?"),
    "WC_meeting_CE_mission":("categorical", "1=No, 2=Somewhat, 3=Yes", "Is WC meeting CE mission?"),
    "WC_CE_resources":    ("categorical", "1=No, 2=Somewhat, 3=Yes", "Does WC provide CE resources?"),
    "Overall_CE_rating":  ("Likert-5", "1=Not at all engaged…5=Extremely engaged", "Self-rated overall CE"),
    "Grad_year":          ("numeric", "4-digit year (2023–2030)", "Anticipated graduation year"),
    "Major_raw":          ("text", "Open response", "Raw major text entry — see Major1/Major2"),
    "Major1":             ("categorical", "WC 3-letter catalog code (e.g. PSY, ENG)", "Primary major recoded from Major_raw"),
    "Major2":             ("categorical", "WC 3-letter catalog code or ''", "Second major (double major) from Major_raw"),
    "Transfer_student":   ("binary", "1=Yes, 0=No; -9 for SP23", "Transfer student? (asked SP24/SP25 only)"),
    "Athlete":            ("binary", "1=Yes, 0=No", "Campus athlete?"),
    "Greek_life":         ("binary", "1=Yes, 0=No", "Fraternity or sorority member?"),
    "Club_member":        ("binary", "1=Yes, 0=No", "Member of group/club/activity?"),
    "Gender":             ("categorical", "1=Man, 2=Woman, 3=Non-binary, 4=Self-describe, 5=Prefer not", "Gender identity"),
    "Race_ethnicity":     ("multi-select text", "Open response / multi-select", "Racial/ethnic identity"),
    "Participant_code":   ("text", "Open response", "Course-credit participant number"),
    # ── One-hot encoded: CE_define_open ──────────────────────────────────
    "CEdef_voting":            ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Voting"),
    "CEdef_volunteering":      ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Volunteering or community service"),
    "CEdef_talking_community": ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Talking with people in the community"),
    "CEdef_activism":          ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Activism or awareness raising"),
    "CEdef_political":         ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Political engagement"),
    "CEdef_protesting":        ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Protesting or rallying"),
    "CEdef_teaching":          ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Teaching or mentorship"),
    "CEdef_fundraising":       ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Fundraising"),
    "CEdef_arts_culture":      ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Attending community arts and culture events"),
    "CEdef_community_business":("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Contributing to community businesses with a social cause"),
    "CEdef_donations":         ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Making donations to charities"),
    "CEdef_research":          ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Participating in research with community members"),
    "CEdef_service_learning":  ("binary OHE", "1=selected, 0=not selected, -9=missing", "CE definition: Service-learning experiences with a class"),
    # ── One-hot encoded: Media_CE_activities ──────────────────────────────
    "Media_repost_events":    ("binary OHE", "1=selected, 0=not selected, -9=missing", "Media CE: Reposting information about current events"),
    "Media_sign_petition":    ("binary OHE", "1=selected, 0=not selected, -9=missing", "Media CE: Signing online petitions"),
    "Media_follow_news":      ("binary OHE", "1=selected, 0=not selected, -9=missing", "Media CE: Following accounts/pages about current events"),
    "Media_follow_officials": ("binary OHE", "1=selected, 0=not selected, -9=missing", "Media CE: Following elected officials on social media"),
    "Media_debate_opinions":  ("binary OHE", "1=selected, 0=not selected, -9=missing", "Media CE: Debating opinions with others who disagree"),
    "Media_like_posts":       ("binary OHE", "1=selected, 0=not selected, -9=missing", "Media CE: Liking posts about current events"),
    "Media_post_opinions":    ("binary OHE", "1=selected, 0=not selected, -9=missing", "Media CE: Posting own opinions about current events"),
    # ── One-hot encoded: Local_news_which ────────────────────────────────
    "News_The_Elm":              ("binary OHE", "1=selected, 0=not selected, -9=missing", "Local news: The Elm"),
    "News_Kent_County":          ("binary OHE", "1=selected, 0=not selected, -9=missing", "Local news: Kent County News"),
    "News_Chestertown_Spy":      ("binary OHE", "1=selected, 0=not selected, -9=missing", "Local news: The Chestertown Spy"),
    "News_Eastern_Shore_Post":   ("binary OHE", "1=selected, 0=not selected, -9=missing", "Local news: Eastern Shore Post"),
    "News_Shore_Daily":          ("binary OHE", "1=selected, 0=not selected, -9=missing", "Local news: Shore Daily News"),
    "News_Delmarva_Now":         ("binary OHE", "1=selected, 0=not selected, -9=missing", "Local news: Delmarva Now"),
}

codebook_rows = []
for col in merged.columns:
    meta = CODEBOOK_META.get(col, ("text", "", ""))
    dtype, valid_vals, notes = meta
    # Sample unique non-missing values
    sample = merged[col][merged[col] != -9].dropna().unique()[:8].tolist()
    codebook_rows.append({
        "Variable":       col,
        "ImportId":       CANONICAL_INV.get(col, "—"),
        "Original_cols":  get_orig_cols(col),
        "Question_text":  get_question_text(col)[:200],
        "Data_type":      dtype,
        "Valid_values":   valid_vals,
        "Missing_code":   -9,
        "Notes":          notes,
        "Sample_values":  str(sample)[:120],
    })

codebook = pd.DataFrame(codebook_rows)

CODEBOOK_EXCEL = "CE_longitudinal_codebook.xlsx"
with pd.ExcelWriter(CODEBOOK_EXCEL, engine="openpyxl") as writer:
    codebook.to_excel(writer, index=False, sheet_name="Codebook")
    # Auto-fit column widths
    ws = writer.sheets["Codebook"]
    for col_cells in ws.columns:
        max_len = max(len(str(c.value or "")) for c in col_cells)
        ws.column_dimensions[col_cells[0].column_letter].width = min(max_len + 2, 60)

print(f"✓ Codebook saved: {CODEBOOK_EXCEL}")
print(f"  {len(codebook)} variables documented")
codebook[["Variable","Data_type","Valid_values","Notes"]].head(20)


✓ Codebook saved: CE_longitudinal_codebook.xlsx
  91 variables documented


,Variable,Data_type,Valid_values,Notes
0,survey_year,string,SP23 / SP24 / SP25,Added: identifies data source year
1,ResponseId,text,,
2,StartDate,text,,
3,RecordedDate,text,,
4,Status,text,,
5,Finished,text,,
6,Duration_sec,text,,
7,Progress,text,,
8,DistributionChannel,text,,
9,Grad_year,numeric,4-digit year (2023–2030),Anticipated graduation year
